In [1]:
#x01_populationStatistics

In [2]:
import matplotlib.pyplot as plt
import uuid
import pdb
import torch
from matplotlib.animation import FuncAnimation
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import gymnasium as gym
import numpy as np
device='cpu'
from torch.utils.tensorboard import SummaryWriter



import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv


from chewc.sim import *
from chewc.callback import *
from chewc.policy import *
from chewc.lab import *

In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv
import random
import numpy as np

# Example usage
n_chr = 1
n_parents = 20
n_loci = 500
founder_pop_size = 500
h2 = 1
reps = 1
max_generations = 10
seed = 422  # Choose any integer for your seed
G = Genome(n_chr, n_loci, seed=seed)



#set up the initial population

founder_haplotypes = np.load('./data/g2f_ch10.npy') 
random_parent_indicies =  np.random.choice(founder_haplotypes.shape[0], n_parents, replace=False)
random_loci_indices = np.random.choice(founder_haplotypes.shape[2], n_loci, replace=False)
founder_haplotypes = founder_haplotypes[random_parent_indicies,:,:]
founder_haplotypes = founder_haplotypes[:,:,random_loci_indices]
founder_haplotypes = torch.tensor(founder_haplotypes)
founder_haplotypes = founder_haplotypes.unsqueeze(2) 


founder_pop = create_pop(G, founder_haplotypes)
founder_pop = random_crosses(founder_pop.haplotypes,founder_pop_size)
founder_pop = create_pop(G, founder_pop)



T = Trait(G, founder_pop, target_mean=0.0, target_variance=1.0, seed=seed)
SP = SimParams(G, T, h2, reps, founder_pop_size, max_generations, founder_pop, seed=seed)


# Create your custom callbacks
genetic_variance_callback = AverageFinalGenerationCallback(log_freq=100)
action_callback = ActionTrackingCallback(log_freq=10)

# Combine the callbacks using CallbackList
combined_callbacks = CallbackList([genetic_variance_callback, action_callback])

# Create the environment
config = {'sparse_reward':True}
env = SelectionIntensityEnvironment(SP, config)
x = env.reset()

NameError: name 'phenotype' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Convert torch tensor to numpy array if necessary
phenotypes = env.population.breeding_values.numpy()

# Calculate summary statistics
mean_phenotype = np.mean(phenotypes)
median_phenotype = np.median(phenotypes)
std_phenotype = np.std(phenotypes)
min_phenotype = np.min(phenotypes)
max_phenotype = np.max(phenotypes)
sum_phenotype = np.sum(phenotypes)

# Print summary statistics
print("Summary Statistics:")
print(f"Mean: {mean_phenotype:.2f}")
print(f"Median: {median_phenotype:.2f}")
print(f"Standard Deviation: {std_phenotype:.2f}")
print(f"Minimum: {min_phenotype:.2f}")
print(f"Maximum: {max_phenotype:.2f}")
print(f"Sum: {sum_phenotype:.2f}")

# Create histogram
plt.figure(figsize=(10, 6))
plt.hist(phenotypes, bins=10, edgecolor='black')
plt.title('Histogram of Phenotypes')
plt.xlabel('Phenotype Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming SimulatedEnv and Population classes are already defined as in your code

def collect_baselines(env, actions, repetitions=100, cycles=5):
    results = {action: {'max_phenotype': [], 'gv': []} for action in actions}
    
    for action in actions:
        for _ in range(repetitions):
            env.reset()
            cycle_max_phenotype = []
            cycle_gv = []
            max_phenotype = env.population.breeding_values.max()
            gv = env.population.breeding_values.var()
            cycle_max_phenotype.append(max_phenotype)
            cycle_gv.append(gv)
            for _ in range(cycles):
                env.step(np.array(action))
                max_phenotype = env.population.breeding_values.max()
                gv = env.population.breeding_values.var()
                cycle_max_phenotype.append(max_phenotype)
                cycle_gv.append(gv)
            
            results[action]['max_phenotype'].append(cycle_max_phenotype)
            results[action]['gv'].append(cycle_gv)
    
    return results

def plot_results(results, metric):
    plt.figure(figsize=(10, 6))
    
    for action, data in results.items():
        mean_values = np.mean(data[metric], axis=0)
        std_values = np.std(data[metric], axis=0)
        cycles = range(1, len(mean_values) + 1)
        
        plt.plot(cycles, mean_values, label=f'Action {action}')
        plt.fill_between(cycles, mean_values - std_values, mean_values + std_values, alpha=0.3)
    
    plt.xlabel('Cycle')
    plt.ylabel(metric.replace('_', ' ').title())
    plt.title(f'{metric.replace("_", " ").title()} over Cycles')
    plt.legend()
    plt.grid(True)
    plt.show()

# Set up the environment and actions
actions = [.05, .5, .95 ]
actions = [scale_values(x, to_range=(-1, 1), from_range=(env.action_low, env.action_high)) for x in actions]
print(actions)
# Collect baselines
results = collect_baselines(env, actions, repetitions=30,cycles=SP.max_generations)

# Plot results
plot_results(results, 'max_phenotype')
plot_results(results, 'gv')
print(actions)

In [ ]:
env.reset()

In [ ]:
import torch
# Wrap the environment (required for SB3)
env.reset()
env = DummyVecEnv([lambda: env])

# Create and train the model with the custom policy
model = PPO(CustomActorCriticPolicy, env, verbose=1, tensorboard_log="./ppotb")
model.learn(total_timesteps=5000, callback=combined_callbacks)

In [ ]:

# # Create the environment
# config = {}
# env = SelectionIntensityEnvironment(SP, config)

# # Wrap the environment (required for SB3)
# env = DummyVecEnv([lambda: env])

# # Create and train the model with the custom policy
# model = PPO(CustomActorCriticPolicy, env, verbose=1, tensorboard_log="./ppotb")
# model.learn(total_timesteps=100000, callback=combined_callbacks)

In [ ]:
SP.max_generations